In [174]:
import pandas as pd
import numpy as np

In [175]:
df = pd.read_csv(r'DDW-C18-0000.csv') # READING C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX
df = df[5:]  # STRIPPING UNWANTED ROWS
df.drop(df.columns[[1, 6, 7, 9, 10]], axis = 1, inplace = True)   # DROPPING UNWANTED COLUMNS
# RENAMING COLUMNS WITH SUITABLE NAMES
df.columns = ['state/code','AreaName','Total/Rural/Urban','Age-group','Persons-Numberspeakingsecondlanguage','Persons-Numberspeakingthirdlanguage']

In [176]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
df = df[df['Total/Rural/Urban']=='Total']
df = df[df['Age-group']=='Total']

In [177]:
df = df.drop(['Total/Rural/Urban','Age-group'], axis = 1)

In [178]:
df = df.drop(df[df['AreaName']=="INDIA"].index)

In [179]:
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
df = df.astype({'Persons-Numberspeakingsecondlanguage':int,'Persons-Numberspeakingthirdlanguage':int})

In [180]:
# COMPUTING PEOPLE SPEAKING EXACTLY TWO LANGUAGES
df['exactlytwo'] = df['Persons-Numberspeakingsecondlanguage'] - df['Persons-Numberspeakingthirdlanguage']

In [181]:
# RATIO OF 3+ TO 2 LANGUAGE SPEAKERS
df['threetotwo'] = df['Persons-Numberspeakingthirdlanguage']/df['exactlytwo']

In [182]:
df.head()

,state/code,AreaName,Persons-Numberspeakingsecondlanguage,Persons-Numberspeakingthirdlanguage,exactlytwo,threetotwo
35,01,JAMMU & KASHMIR,6183190,2096220,4086970,0.512903
65,02,HIMACHAL PRADESH,1242705,347286,895419,0.387847
95,03,PUNJAB,13035224,7829913,5205311,1.504216
125,04,CHANDIGARH,579920,321979,257941,1.248266
155,05,UTTARAKHAND,1757756,187822,1569934,0.119637


In [183]:
# TOP 3 STATES
best = df.sort_values(by=['threetotwo'], ascending=False)[0:3]

In [184]:
# WORST 3 STATES
worst = df.sort_values(by=['threetotwo'], ascending=True)[0:3]

In [185]:
# CONCATENATING TOP AND WORST THREE
df_3to2 = pd.concat([best, worst], ignore_index=True)

In [186]:
df_3to2 = df_3to2[['AreaName','threetotwo']]
df_3to2.rename(columns = {'AreaName':'state-name'}, inplace = True)

In [187]:
df_3to2

,state-name,threetotwo
0,GOA,1.925807
1,PUNJAB,1.504216
2,CHANDIGARH,1.248266
3,BIHAR,0.111582
4,UTTARAKHAND,0.119637
5,UTTAR PRADESH,0.127448


In [188]:
dfp = pd.read_csv(r'DDW_PCA0000_2011_Indiastatedist.csv') # READING INDIA CENSUS DETAILS
dfp = dfp.iloc[:,6:13] # Extracting only required information from dataframe

In [189]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
dfp = dfp[dfp['TRU'] == 'Total']
dfp = dfp.drop(['TRU','No_HH','TOT_M','TOT_F'], axis = 1)

In [190]:
dfp_state = dfp[dfp['Level']=='STATE']
dfp_state = dfp_state.drop(['Level'],axis=1)

In [191]:
dfp_state.rename(columns = {'Name':'AreaName'}, inplace = True)

In [192]:
# MERGING BOTH DATA FRAMES
df_overall = pd.merge(df, dfp_state, on='AreaName')

In [193]:
# COMPUTING PEOPLE SPEAKING EXACTLY ONE LANGUAGE
df_overall['exactlyone'] = df_overall['TOT_P'] - df_overall['Persons-Numberspeakingsecondlanguage']

In [194]:
# RATIO OF 2 TO 1
df_overall['twotoone'] = df_overall['exactlytwo']/df_overall['exactlyone']

In [195]:
df_overall.head()

,state/code,AreaName,Persons-Numberspeakingsecondlanguage,Persons-Numberspeakingthirdlanguage,exactlytwo,threetotwo,TOT_P,exactlyone,twotoone
0,01,JAMMU & KASHMIR,6183190,2096220,4086970,0.512903,12541302,6358112,0.642796
1,02,HIMACHAL PRADESH,1242705,347286,895419,0.387847,6864602,5621897,0.159273
2,03,PUNJAB,13035224,7829913,5205311,1.504216,27743338,14708114,0.353907
3,04,CHANDIGARH,579920,321979,257941,1.248266,1055450,475530,0.542428
4,05,UTTARAKHAND,1757756,187822,1569934,0.119637,10086292,8328536,0.188501


In [196]:
# COMPUTING BEST AND WORST AND THEN APPENDING BOTH AS WE DID EARLIER
best = df_overall.sort_values(by=['twotoone'], ascending=False)[0:3]
worst = df_overall.sort_values(by=['twotoone'], ascending=True)[0:3]
df_2to1 = pd.concat([best, worst], ignore_index=True)
df_2to1 = df_2to1[['AreaName','twotoone']]
df_2to1.rename(columns = {'AreaName':'state-name'}, inplace = True)

In [197]:
df_2to1

,state-name,twotoone
0,ANDAMAN & NICOBAR ISLANDS,1.499862
1,GOA,1.157976
2,ARUNACHAL PRADESH,0.938953
3,RAJASTHAN,0.105877
4,UTTAR PRADESH,0.114661
5,CHHATTISGARH,0.133643


In [198]:
# WRITING FINAL DATAFRAMES INTO CSV FILE
df_3to2.to_csv (r'3-to-2-ratio.csv', index = None, header=True)
df_2to1.to_csv (r'2-to-1-ratio.csv', index = None, header=True)